# TFM

##1. Configuacion

In [1]:
# Instalar librerias
!pip install pyeph > /dev/null 2>&1
!pip install scikit-learn openpyxl > /dev/null 2>&1

In [2]:
# Importar librerias
import pyeph
import pandas as pd
import numpy as np

from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.svm import SVC

##2. Extraer bases de datos
Se utiliza la libreria [pyeph](https://pypi.org/project/pyeph/)

In [3]:
# Diccionarios para guardar las bases
bases_individuales = {}
bases_hogar = {}

# Descargar datos para cada año y trimestre
for ano in range(2023, 2024):  # Años 2023 a 2023
    for trimestre in range(1, 5):  # Trimestres 1 a 4
        clave = f"{ano}T{trimestre}"

        # Base individual
        try:
            df_ind = pyeph.obtener(data="eph", ano=ano, periodo=trimestre, tipo_base="individual")
            bases_individuales[clave] = df_ind
            print(f"✓ Base individual {clave} descargada")
        except Exception as e:
            print(f"✗ Error al descargar base individual {clave}: {e}")

        # Base hogar
        try:
            df_hogar = pyeph.obtener(data="eph", ano=ano, periodo=trimestre, tipo_base="hogar")
            bases_hogar[clave] = df_hogar
            print(f"✓ Base hogar {clave} descargada")
        except Exception as e:
            print(f"✗ Error al descargar base hogar {clave}: {e}")

Obtenido con exito: base_individual_2023T1.zip 
✓ Base individual 2023T1 descargada
Obtenido con exito: base_hogar_2023T1.zip 
✓ Base hogar 2023T1 descargada
Obtenido con exito: base_individual_2023T2.zip 
✓ Base individual 2023T2 descargada
Obtenido con exito: base_hogar_2023T2.zip 
✓ Base hogar 2023T2 descargada
Obtenido con exito: base_individual_2023T3.zip 
✓ Base individual 2023T3 descargada
Obtenido con exito: base_hogar_2023T3.zip 
✓ Base hogar 2023T3 descargada
Obtenido con exito: base_individual_2023T4.zip 
✓ Base individual 2023T4 descargada
Obtenido con exito: base_hogar_2023T4.zip 
✓ Base hogar 2023T4 descargada


In [ ]:
# Exportar a Excel las bases del 4T2023
#clave = "2023T4"

#bases_individuales[clave].to_excel(f"base_individual_{clave}.xlsx", index=False)

#bases_hogar[clave].to_excel(f"base_hogar_{clave}.xlsx", index=False)

##3. Procesar datos

###3.1 Crear bases

In [49]:
# Clave del trimestre
clave = "2023T4"

# Variables necesarias desde la base hogar
variables_hogar_interes = [
    'CODUSU', 'NRO_HOGAR', 'ANO4', 'TRIMESTRE', 'REGION', 'MAS_500', 'AGLOMERADO',
    'PONDERA', 'PONDIH', 'IX_TOT', 'IPCF',
    'IV1', 'IV2', 'IV3', 'IV4', 'IV5', 'IV6', 'IV7', 'IV8', 'IV9', 'IV10', 'IV11',
    'IV12_1', 'IV12_2', 'IV12_3',
    'II1', 'II2', 'II3', 'II4_1', 'II4_2', 'II4_3', 'II7', 'II8', 'II9',
    'V1', 'V2', 'V7', 'V8', 'V9', 'V10', 'V19_A', 'V19_B',
    'VII1_1', 'VII1_2', 'VII2_1', 'VII2_2', 'VII2_3', 'VII2_4'
]

# Cargar base
df_hogar_interes = bases_hogar[clave][variables_hogar_interes].copy()

# Variables finales que se conservarán
variables_filtradas = [
    'CODUSU', 'PONDERA', 'IV1', 'IV2', 'IV3', 'IV4', 'IV5', 'IV6', 'IV7',
    'IV8', 'IV9', 'IV10', 'IV11', 'IV12_1', 'IV12_2', 'IV12_3',
    'II1', 'II2', 'II3', 'II4_1', 'II4_2', 'II4_3', 'II7', 'II8', 'II9',
    'V1', 'V2', 'V7', 'V8', 'V9', 'V10', 'V19_A', 'V19_B',
    'IX_TOT', 'IPCF', 'PONDIH',
    'VII1_1', 'VII1_2', 'VII2_1', 'VII2_2', 'VII2_3', 'VII2_4'
]

# Subconjunto del DataFrame
df_hogar_limpio = df_hogar_interes[variables_filtradas].copy()

# Corregir formato de IPCF (coma decimal → punto)
df_hogar_limpio['IPCF'] = df_hogar_limpio['IPCF'].astype(str).str.replace(",", ".")
df_hogar_limpio['IPCF'] = pd.to_numeric(df_hogar_limpio['IPCF'], errors='coerce')

# Corregir valores 0 en variables VII*
variables_vii = ['VII1_1', 'VII1_2', 'VII2_1', 'VII2_2', 'VII2_3', 'VII2_4']
df_hogar_limpio[variables_vii] = df_hogar_limpio[variables_vii].replace(0, 98)

# Convertir a numérico las columnas que lo requieran
columnas_excluir_conversion = ['CODUSU']
for col in df_hogar_limpio.columns:
    if col not in columnas_excluir_conversion:
        df_hogar_limpio[col] = pd.to_numeric(df_hogar_limpio[col], errors='coerce')

# Rangos válidos por variable
rangos_validos = {
    'IV1': (1, 5), 'IV2': (1, 10), 'IV3': (1, 3), 'IV4': (1, 9), 'IV5': (1, 2),
    'IV6': (1, 3), 'IV7': (1, 3), 'IV8': (1, 2), 'IV9': (1, 3), 'IV10': (1, 3), 'IV11': (1, 4),
    'IV12_1': (1, 2), 'IV12_2': (1, 2), 'IV12_3': (1, 2),
    'II1': (1, 10), 'II2': (1, 10), 'II3': (1, 2),
    'II4_1': (1, 2), 'II4_2': (1, 2), 'II4_3': (1, 2),
    'II7': (1, 8), 'II8': (1, 3), 'II9': (1, 4),
    'V1': (1, 2), 'V2': (1, 2), 'V7': (1, 2), 'V8': (1, 2),
    'V9': (1, 2), 'V10': (1, 2), 'V19_A': (1, 2), 'V19_B': (1, 2),
    'IX_TOT': (1, 20), 'IPCF': (0, 5000000),
    'VII1_1': (1, 50), 'VII1_2': (1, 50),
    'VII2_1': (1, 50), 'VII2_2': (1, 50), 'VII2_3': (1, 50), 'VII2_4': (1, 50)
}

# Valores especiales permitidos en variables VII
valores_extra_vii = [96, 97, 98]

# Aplicar filtros por rango
for var, (min_val, max_val) in rangos_validos.items():
    if var in variables_vii:
        df_hogar_limpio = df_hogar_limpio[df_hogar_limpio[var].isin(
            list(range(min_val, max_val + 1)) + valores_extra_vii)]
    else:
        df_hogar_limpio = df_hogar_limpio[
            (df_hogar_limpio[var] >= min_val) & (df_hogar_limpio[var] <= max_val)
        ]

# Exportar a Excel (descomentar para guardar archivo)
# df_hogar_limpio.to_excel("hogar_limpio_2023T4.xlsx", index=False)

# Mostrar DataFrame final
df_hogar_limpio

,CODUSU,PONDERA,IV1,IV2,IV3,IV4,IV5,IV6,IV7,IV8,...,V19_B,IX_TOT,IPCF,PONDIH,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4
0,TQRMNORSQHKMKUCDEIGED00791045,107,1,4,1,1,1,1,1,1,...,2,3,283333.33,128,1,98,98,98,98,98
1,TQRMNOPVSHJMLNCDEIGED00801502,137,1,3,1,1,2,1,1,1,...,2,2,0.00,0,1,98,2,98,98,98
2,TQRMNORTYHMLKPCDEHJGH00810920,356,2,4,1,9,1,1,1,1,...,2,4,192500.00,329,2,98,1,98,98,98
3,TQRMNOQTWHKLKQCDEHJGH00790284,299,1,6,1,3,1,1,1,1,...,2,3,33333.33,320,1,98,2,98,98,98
4,TQRMNORQVHLLKRCDEHJGH00853836,221,1,5,1,2,1,1,1,1,...,2,2,150000.00,193,1,98,96,98,98,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16456,TQRMNOSXVHKMLMCDEFMDB00787973,180,1,4,1,1,1,1,1,1,...,2,4,30000.00,322,1,98,2,3,4,98
16457,TQRMNOQQYHLOLMCDEFMDB00852214,170,2,3,1,1,1,1,1,1,...,2,2,0.00,0,2,98,98,98,98,98
16458,TQRMNOUQYHLOLNCDEFMDB00852215,249,1,2,1,4,2,1,1,1,...,2,3,0.00,0,1,98,98,98,98,98
16460,TQRMNOSXQHJMKQCDEGKDB00794419,92,1,5,1,4,1,1,1,1,...,2,12,0.00,0,1,7,8,3,98,98


In [ ]:
# Definir la clave correspondiente al cuarto trimestre de 2023
clave = "2023T4"

# Variables de interés a nivel individuo (extensiva pero relevante)
variables_individuo_interes = [
    # Identificación y estructura
    'CODUSU', 'ANO4', 'TRIMESTRE', 'NRO_HOGAR', 'COMPONENTE',
    'REGION', 'MAS_500', 'AGLOMERADO', 'PONDERA', 'PONDIH',

    # Demográficas
    'CH03', 'CH04', 'CH05', 'CH06', 'CH07',

    # Cobertura de salud y alfabetización
    'CH08', 'CH09',

    # Variables educativas clave
    'CH10', 'CH11', 'CH12', 'CH13', 'CH14', 'NIVEL_ED',

    # Migración
    'CH15', 'CH15_COD', 'CH16', 'CH16_COD',

    # Condición de actividad
    'ESTADO', 'CAT_OCUP', 'CAT_INAC', 'IMPUTA',

    # Búsqueda de empleo
    'PP02C1', 'PP02C2', 'PP02C3', 'PP02C4', 'PP02C5', 'PP02C6',
    'PP02C7', 'PP02C8', 'PP02E', 'PP02H', 'PP02I',

    # Ocupación actual
    'PP03C', 'PP03D', 'PP3E_TOT', 'PP3F_TOT',
    'PP03G', 'PP03H', 'PP03I', 'PP03J',
    'INTENSI',

    # Información del empleo actual
    'PP04A', 'PP04B_COD', 'PP04D_COD', 'PP04G',
    'PP05B2_MES', 'PP05B2_ANO', 'PP05C_1', 'PP05C_2', 'PP05C_3', 'PP05F',

    # Condiciones laborales (asalariados)
    'PP07A', 'PP07C', 'PP07D', 'PP07E', 'PP07F1', 'PP07F2', 'PP07F3',
    'PP07F4', 'PP07F5', 'PP07G1', 'PP07G2', 'PP07G3', 'PP07G4', 'PP07G_59',
    'PP07H', 'PP07I', 'PP07J', 'PP07K',

    # Ingresos laborales y totales
    'P21', 'TOT_P12', 'P47T',
    'DECINDR', 'IDECINDR', 'RDECINDR', 'GDECINDR', 'PDECINDR', 'ADECINDR',
    'PONDIIO', 'PONDII',

    # Ingresos no laborales
    'V2_M', 'V3_M', 'V4_M', 'V5_M', 'V8_M', 'V9_M', 'V10_M', 'V11_M',
    'V12_M', 'V18_M', 'V19_AM', 'V21_M', 'T_VI',

    # Ingresos familiares desde individuo (puede usarse para agregar)
    'ITF', 'IPCF', 'DECIFR', 'DECCFR', 'PONDIH'
]

# Crear el nuevo DataFrame filtrado con las columnas deseadas
df_individuo_interes = bases_individuales[clave][variables_individuo_interes].copy()

# Mostrar primeras filas del DataFrame de interés
df_individuo_interes.head()

###3.2 Conformacion base de interes

In [ ]:
# Asegurar tipos numéricos
df_individuo_interes['CH06'] = pd.to_numeric(df_individuo_interes['CH06'], errors='coerce')
df_individuo_interes['CH14'] = pd.to_numeric(df_individuo_interes['CH14'], errors='coerce')

# Calcular escolaridad esperada (máximo 15 años desde los 5 años de edad)
df_individuo_interes['escolaridad_esperada'] = (df_individuo_interes['CH06'] - 5).clip(lower=0, upper=15)
df_individuo_interes['escolaridad_completada'] = df_individuo_interes['CH14']

# Índice de Escolaridad Ajustado por Edad (IEAE)
df_individuo_interes['IEAE'] = df_individuo_interes['escolaridad_completada'] / df_individuo_interes['escolaridad_esperada']
df_individuo_interes.loc[df_individuo_interes['escolaridad_esperada'] == 0, 'IEAE'] = pd.NA

# Filtrar individuos válidos para IEAE
df_IEAE_validos = df_individuo_interes[(df_individuo_interes['CH06'] > 6) & (df_individuo_interes['IEAE'].notnull())]

# Calcular NE por hogar (media simple)
df_NE = df_IEAE_validos.groupby(['CODUSU', 'NRO_HOGAR'], as_index=False).agg(
    NE=('IEAE', 'mean'),
    N=('COMPONENTE', 'count')
)

# Unir NE al DataFrame de hogares
df_hogar_interes = df_hogar_interes.merge(df_NE, on=['CODUSU', 'NRO_HOGAR'], how='left')

# Crear variable de hacinamiento: personas por cuarto
df_hogar_interes['hacinamiento'] = df_hogar_interes.apply(
    lambda row: row['IX_TOT'] / row['IV2'] if pd.notnull(row['IX_TOT']) and pd.notnull(row['IV2']) and row['IV2'] > 0 else np.nan,
    axis=1
)


# Crear variables binarias para proporciones
df_individuo_interes['es_mujer'] = df_individuo_interes['CH04'] == 2
df_individuo_interes['es_analfabeto'] = df_individuo_interes['CH09'] == 2
df_individuo_interes['es_desocupado'] = df_individuo_interes['ESTADO'] == 2

# Agregar proporciones por hogar (sin ponderar)
df_agregado = df_individuo_interes.groupby(['CODUSU', 'NRO_HOGAR'], as_index=False).agg(
    proporcion_mujeres=('es_mujer', 'mean'),
    promedio_edad=('CH06', 'mean'),
    proporcion_analfabetismo=('es_analfabeto', 'mean'),
    proporcion_desocupados=('es_desocupado', 'mean')
)

# Unir agregados al DataFrame de hogares
df_hogar_interes = df_hogar_interes.merge(df_agregado, on=['CODUSU', 'NRO_HOGAR'], how='left')

# Guardar base procesada
df_hogar_interes.to_csv("hogares_2023T4.csv", index=False)
df_hogar_interes.to_excel("hogares_2023T4.xlsx", index=False)

print(df_hogar_interes.columns.tolist())

##4. Regresion
$${NE}_i = f(X_i) + e_i$$

###4.1 MCO con seleccion de variables

In [ ]:
# Preparar datos para la regresión

# Filtrar hogares con NE definido
df_modelo = df_hogar_interes[df_hogar_interes['NE'].notnull()].copy()

# Variables explicativas (X)
X = df_modelo[[
    'hacinamiento',
    'ITF', 'IPCF',
    'proporcion_mujeres',
    'promedio_edad',
    'proporcion_analfabetismo',
    'proporcion_desocupados'
]].copy()

# Limpiar comas decimales y convertir a float
for col in X.columns:
    X[col] = X[col].astype(str).str.replace(",", ".", regex=False)
    X[col] = pd.to_numeric(X[col], errors="coerce")

# Variable dependiente
y = df_modelo['NE']

# Eliminar filas con valores faltantes
df_modelo_limpio = pd.concat([X, y], axis=1).dropna()
X = df_modelo_limpio.drop(columns='NE')
y = df_modelo_limpio['NE']

In [ ]:
# Crear y entrenar modelo con Elastic Net

# Separar conjunto de entrenamiento y testeo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar pipeline con estandarización + ElasticNetCV
modelo = make_pipeline(
    StandardScaler(),
    ElasticNetCV(cv=5, l1_ratio=[.1, .5, .7, .9, .95, 1], random_state=42)
)

modelo.fit(X_train, y_train)

In [ ]:
# Evaluar el modelo

y_pred = modelo.predict(X_test)
print("R²:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

# Mostrar coeficientes
coefs = modelo.named_steps['elasticnetcv'].coef_
for var, coef in zip(X.columns, coefs):
    print(f"{var}: {coef:.4f}")

In [ ]:
# Mostrar los coeficientes seleccionados

# Obtener coeficientes del modelo entrenado
coefs = modelo.named_steps['elasticnetcv'].coef_
variables = X.columns

print("Coeficientes del modelo (Elastic Net):")
for var, coef in zip(variables, coefs):
    print(f"{var}: {coef:.4f}")

###4.2 Bosques aleatorios

In [ ]:
# Preparar datos

# Filtrar hogares con NE definido
df_modelo = df_hogar_interes[df_hogar_interes['NE'].notnull()].copy()

# Variables independientes (X)
X = df_modelo[[
    'hacinamiento',
    'ITF', 'IPCF',
    'proporcion_mujeres',
    'promedio_edad',
    'proporcion_analfabetismo',
    'proporcion_desocupados'
]].copy()

# Convertir comas a puntos y asegurar tipo float
for col in X.columns:
    X[col] = X[col].astype(str).str.replace(",", ".", regex=False)
    X[col] = pd.to_numeric(X[col], errors='coerce')

# Variable dependiente
y = df_modelo['NE']

# Eliminar filas con valores faltantes
df_modelo_limpio = pd.concat([X, y], axis=1).dropna()
X = df_modelo_limpio.drop(columns='NE')
y = df_modelo_limpio['NE']

In [ ]:
# Entrenar modelo

# Dividir datos en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y ajustar el modelo
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

In [ ]:
# Evaluar modelo

# Predicciones
y_pred = rf_model.predict(X_test)

# Métricas
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"R² del modelo: {r2:.4f}")
print(f"RMSE del modelo: {rmse:.4f}")

In [ ]:
# Relevancia de variables

# Mostrar importancia de variables
importancia = rf_model.feature_importances_

print("\nRelevancia de variables:")
for var, imp in sorted(zip(X.columns, importancia), key=lambda x: x[1], reverse=True):
    print(f"{var}: {imp:.4f}")

###4.3 Support Vector Regressor (SVR)

In [ ]:
# Preparar datos

# Filtrar hogares con NE definido
df_modelo = df_hogar_interes[df_hogar_interes['NE'].notnull()].copy()

# Variables independientes (X)
X = df_modelo[[
    'hacinamiento',
    'ITF', 'IPCF',
    'proporcion_mujeres',
    'promedio_edad',
    'proporcion_analfabetismo',
    'proporcion_desocupados'
]].copy()

# Corregir comas y convertir a float
for col in X.columns:
    X[col] = X[col].astype(str).str.replace(",", ".", regex=False)
    X[col] = pd.to_numeric(X[col], errors='coerce')

# Variable dependiente
y = df_modelo['NE']

# Eliminar filas con valores faltantes
df_modelo_limpio = pd.concat([X, y], axis=1).dropna()
X = df_modelo_limpio.drop(columns='NE')
y = df_modelo_limpio['NE']

In [ ]:
# Entrenar modelo

# Dividir en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear pipeline: escalado + SVR
svr_model = make_pipeline(
    StandardScaler(),
    SVR(kernel='rbf', C=1.0, epsilon=0.1)
)

# Entrenar modelo
svr_model.fit(X_train, y_train)

In [ ]:
# Evaluar modelo

# Predicciones
y_pred = svr_model.predict(X_test)

# Métricas
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"R² del modelo SVR: {r2:.4f}")
print(f"RMSE del modelo SVR: {rmse:.4f}")

##5. Clasificacion
$$PE_i = \mathbb{I}({NE}_i<\tau)$$

$\tau$: Mediana

###5.1 Bosques aleatorios

In [ ]:
# Crear variable PE_i

# Usar hogares con NE definido
df_clasificacion = df_hogar_interes[df_hogar_interes['NE'].notnull()].copy()

# Calcular mediana de NE
tau = df_clasificacion['NE'].median()

# Crear variable binaria: 1 si NE < mediana (pobreza educativa)
df_clasificacion['PE_i'] = (df_clasificacion['NE'] < tau).astype(int)

In [ ]:
# Preparar datos

# Variables explicativas
X = df_clasificacion[[
    'hacinamiento',
    'ITF', 'IPCF',
    'proporcion_mujeres',
    'promedio_edad',
    'proporcion_analfabetismo',
    'proporcion_desocupados'
]].copy()

# Corregir comas y convertir a float
for col in X.columns:
    X[col] = X[col].astype(str).str.replace(",", ".", regex=False)
    X[col] = pd.to_numeric(X[col], errors='coerce')

# Variable dependiente
y = df_clasificacion['PE_i']

# Eliminar filas con valores faltantes
df_clasificacion_limpio = pd.concat([X, y], axis=1).dropna()
X = df_clasificacion_limpio.drop(columns='PE_i')
y = df_clasificacion_limpio['PE_i']

In [ ]:
# Entrenar modelo

# Separar en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar modelo
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

In [ ]:
# Evaluar modelo

# Predicciones
y_pred = clf.predict(X_test)

# Métricas de evaluación
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred))
print("\nMatriz de confusión:\n", confusion_matrix(y_test, y_pred))

In [ ]:
# Relevancia de variables

print("\nRelevancia de variables:")
for var, imp in sorted(zip(X.columns, clf.feature_importances_), key=lambda x: x[1], reverse=True):
    print(f"{var}: {imp:.4f}")

###5.2 Support Vector Machine (SVM)

In [ ]:
# Entrenar el modelo

# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear pipeline con estandarización y SVM (con kernel RBF)
svm_model = make_pipeline(
    StandardScaler(),
    SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
)

# Entrenar
svm_model.fit(X_train, y_train)

In [ ]:
# Evaluar el modelo

# Predicciones
y_pred = svm_model.predict(X_test)

# Métricas
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred))
print("\nMatriz de confusión:\n", confusion_matrix(y_test, y_pred))